In [ ]:
import boto3
import json
from langchain.tools import tool
from langchain_aws import ChatBedrock
from langchain.chat_models import BedrockChat
from langchain.agents import initialize_agent, AgentType

# ==== AWS Configuration ====
REGION = "eu-west-2"
TARGET_LAMBDA = "test_function_26May2025"
KNOWLEDGE_BASE_ID = "ZTPJ2E7RTK"
MODEL_ARN = "arn:aws:bedrock:eu-west-2::foundation-model/anthropic.claude-3-sonnet-20240229-v1:0"

lambda_client = boto3.client("lambda", region_name=REGION)
brt_client = boto3.client("bedrock-agent-runtime", region_name=REGION)

# ==== TOOL 1: Structured Data Summary via Lambda ====

import json
from langchain.tools import tool

@tool
def structured_summary_tool(input: str) -> str:
    """
    Summarizes structured data using a Lambda function. Expects a JSON string like:
    {"question": "your question", "schema_id": "your_schema_id"}
    """
    data = json.loads(input)
    question = data["question"]
    schema_id = data.get("schema_id", "vitic_lhasa")

    response = lambda_client.invoke(
        FunctionName=TARGET_LAMBDA,
        InvocationType='RequestResponse',
        Payload=json.dumps({
            "question": question,
            "schema_id": schema_id
        })
    )
    payload = json.loads(response["Payload"].read())
    return payload.get("response", "No structured summary returned.")

# ==== TOOL 2: RAG with Bedrock Knowledge Base ====

@tool
def structured_rag_tool(input: str) -> str:
    """
    Calls Amazon Bedrock Knowledge Base RAG using a JSON string input with:
    {
        "question": "...",
        "narrative_response": "..."
    }
    """
    import json
    data = json.loads(input)
    question = data["question"]
    narrative_response = data["narrative_response"]

    combined_input = (
        f"[Structured Database Response]\n"
        f"{narrative_response}\n\n"
        f"[Follow-up Question]\n"
        f"{question}"
    )

    rag_response = brt_client.retrieve_and_generate(
        input={"text": combined_input},
        retrieveAndGenerateConfiguration={
            "type": "KNOWLEDGE_BASE",
            "knowledgeBaseConfiguration": {
                "knowledgeBaseId": KNOWLEDGE_BASE_ID,
                "modelArn": MODEL_ARN
            }
        }
    )

    return rag_response["output"]["text"]

# ==== Initialize LangChain Agent ====

llm = ChatBedrock(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    region_name="eu-west-2"
)

#tools = [structured_summary_tool, structured_rag_tool]

agent = initialize_agent(
    tools=[structured_rag_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [ ]:
summary = structured_summary_tool.run('{"question": "Is Omeprazole carcinogenic?"}')
print("Structured summary:")
print(summary)

response = structured_rag_tool.run(json.dumps({
    "question": "Is Omeprazole carcinogenic",
    "narrative_response": summary
}))
print("\nFinal RAG response:")
print(response)

Structured summary:
**Compound Evaluated**: Omeprazole, COc1ccc2nc([S+]([O-])Cc3ncc(C)c(OC)c3C)[nH]c2c1
---

Number of records analysed: 17. **Conclusion**: Based on the findings, there is evidence that Omeprazole exposure caused increased incidence of malignant carcinoid tumors in the gastric mucosa of rats in two year studies, which was statistically significant. However, no statistically significant increases in tumor incidence were observed in 18-month studies in mice. The reliability of these studies was rated as not assignable. There is no data present in Vitic directly measuring carcinogenicity in humans, but positive evidence from other species and implicated mechanisms of action may have relevance to humans.

**Study Findings**:

---
- In a 2 year study test study, Rats (Sprague-Dawley) exposed to Omeprazole for an exposure duration of 104 weeks with a study duration of 104 weeks via Gavage route, affected Stomach- gastric mucosa showed Tumour- carcinoid- malignant result, with a 4- Not assignable reliability indicating a positive result, the goldtd50 of 1120 as a measure of potency, the td50notes 'Gold calculated the TD50 using the terminal sacrifice (summary) method.', the pvalue of <=0.0005 as a measure of statistical significance.
- In a 2 year study test study, Rats (Sprague-Dawley) exposed to Omeprazole for an exposure duration of 104 weeks with a study duration of 104 weeks via Gavage route, affected Stomach- gastric mucosa showed Tumour- carcinoid- malignant result, with a 4- Not assignable reliability indicating a positive result, the goldtd50 of 63 as a measure of potency, the td50notes 'The Chi2 value indicated the dose-response data showed a poor fit for the TD50 model. Gold removed dose groups prior to TD50 calculation due to the curve of the dose-response data. This was not done when calculating the Lhasa TD50. Gold calculated the TD50 using the terminal sacrifice (summary) method.', the pvalue of <=0.0005 as a measure of statistical significance.
- In an 18 month study test study, Mice (CD-1) exposed to Omeprazole for an exposure duration of 78 weeks with a study duration of 78 weeks via Gavage route, affected Stomach- gastric mucosa showed no statistically significant increase in Tumour- carcinoid- malignant, with a 4- Not assignable reliability, the pvalue of =1.0 indicates this was not statistically significant.
- In an 18 month study test study, Mice (CD-1) exposed to Omeprazole for an exposure duration of 78 weeks with a study duration of 78 weeks via Gavage route, affected Stomach- gastric mucosa showed no statistically significant increase in Tumour- carcinoid- malignant, with a 4- Not assignable reliability, the pvalue of =1.0 indicates this was not statistically significant.

Final RAG response:
The evidence suggests that Omeprazole may be carcinogenic in rats, as it caused a statistically significant increase in malignant carcinoid tumors in the gastric mucosa in two-year rat studies. However, it did not cause a statistically significant increase in tumors in 18-month mouse studies. While the data shows carcinogenic effects in rats, there is no direct evidence provided on carcinogenicity in humans. The search results mention that clinical data and monitoring in humans receiving chronic Omeprazole therapy did not show similar findings of gastric neoplasia seen in rats, suggesting the carcinogenic effects may be specific to rodents.


In [ ]:
inputs = {
    "question": "What is the carcinogenic potential of Omeprazole and compunds with similarity 0.65. Limit 250 records",
    "schema_id": "vitic_lhasa"
}

result = app.invoke(inputs)
print("Final RAG response:\n", result["final_answer"])

[Ontology-Enriched Question]: What is the carcinogenic potential of Omeprazole and compunds with similarity 0.65. Limit 250 records. Related concepts: RECORDS, OMEPRAZOLE

--- Reflector Iteration #1 ---
Verdict: COMPLETED
Justification: The response provides a comprehensive overview of the carcinogenic potential of Omeprazole based on animal studies, including information on the specific types of tumors observed, species differences, and potency estimates. It also acknowledges the lack of direct human data while suggesting potential relevance based on the animal findings and mechanisms of action.


[CITATIONS RETURNED FROM RAG]:

Citation 1:
- URI: s3://lhasalimited-gen-ai-unstructured-data-store-transform/zotero-papers/Computational Toxicology/Stalford et al. - 2024 - Structuring expert review using AOPs Enabling rob.pdf
  Snippet: Table 2 Carcinogenicity data for compounds pharmacologically similar to Lansoprazole.     Similar Compound Carcinogenicity FDA Labels/Reviews     Omeprazole Rat – stomach (gastric mucosa) [21,41,42] G...

Citation 2:
- URI: s3://lhasalimited-gen-ai-unstructured-data-store-transform/pubmed/8741782.pdf
  Snippet: Since the drug is chemically allied to chlorambu- cil, which has leukemogenic properties, one could argue that the drug should now be classified as a human carcinogen based on this one study. On the o...
- URI: s3://lhasalimited-gen-ai-unstructured-data-store-transform/zotero-papers/Frontiers in Toxicology/ftox-06-1377990.pdf
  Snippet: In 1998 findings of osteosarcoma in the rat carcinogenicity study triggered a halt to ongoing clinical trials, and the sponsor Eli Lilly and Co., conducted long term studies in monkeys demonstrating t...
- URI: s3://lhasalimited-gen-ai-unstructured-data-store-transform/pubmed/1795642.txt
  Snippet: Because of its reported ability to induce unscheduled DNA synthesis in the gastric mucosa, the safety of omeprazole, a potentially clinically useful anti-ulcer drug, has been the subject of debate. We...
Metadata fetch failed for Stalford et al. - 2024 - Structuring expert review using AOPs Enabling rob.pdf: An error occurred (NoSuchKey) when calling the GetObject operation: The specified key does not exist.
[DEBUG] Metadata keys for 8741782.pdf: ['secondaryKeywords', 'volume', 'journal', 'pages', 'issue', 'year', 'primaryKeywords', 'pmid', 'title', 'chemicals', 'doi']
Metadata fetch failed for ftox-06-1377990.pdf: An error occurred (NoSuchKey) when calling the GetObject operation: The specified key does not exist.
[DEBUG] Metadata keys for 1795642.txt: ['secondaryKeywords', 'volume', 'journal', 'pages', 'issue', 'year', 'pmid', 'title', 'chemicals', 'doi']

[ENRICHED QUESTION SENT TO RAG]:
[Structured Database Response]
Based on the findings, there is evidence from 2-year and chronic studies in rats that Omeprazole induces malignant carcinoid tumors in the gastric mucosa, with positive results observed in both sexes. The evidence in mice is more equivocal, with some studies showing negative results and others being inconclusive. The potency estimates (TD50 values) suggest Omeprazole is a moderately potent carcinogen in rats. Other proton pump inhibitor drugs like Lansoprazole also showed evidence of carcinogenic activity in rodents. There is no data present in Vitic directly measuring carcinogenicity in humans, but positive evidence from other species and implicated mechanisms of action may have relevance to humans.

[Follow-up Question]
What is the carcinogenic potential of Omeprazole and compunds with similarity 0.65. Limit 250 records. Related concepts: RECORDS, OMEPRAZOLE
[Additional Metadata Context]
- Citation Metadata:
  Title: Nonhormonal drugs and cancer.
  Journal: Environmental health perspectives
  Year: 1995
  Volume: 103 Suppl 8
  Issue: Suppl 8
  Pages: 191-196
  PMID: 8741782
  DOI: 10.1289/ehp.95103s8191
  Chemicals: Carcinogens
  Keywords: Drug-Related Side Effects and Adverse Reactions, Carcinogens, Animals, Humans, Neoplasms
- Citation Metadata:
  Title: Omeprazole: an exploration of its reported genotoxicity.
  Journal: Mutagenesis
  Year: 1991
  Volume: 6
  Issue: 5
  Pages: 381-384
  PMID: 1795642
  DOI: 10.1093/mutage/6.5.381
  Chemicals: Omeprazole, Carcinogens, Mutagens, omeprazole cyclic sulfenamide
  Keywords: Omeprazole, Carcinogens, Animals, Mutagens, Mutagenicity Tests, Rodentia, Software, Carcinogenicity Tests, Databases, Factual
Final RAG response:
 Based on the search results, there is evidence that omeprazole and other proton pump inhibitors like lansoprazole can induce carcinogenic effects in rodents, particularly in rats. Specifically:

- Omeprazole induced stomach (gastric mucosa) tumors in rats (source 2)
- Lansoprazole and other proton pump inhibitors like pantoprazole, esomeprazole, and rabeprazole also showed evidence of inducing stomach tumors in rats (source 2) However, the search results indicate that these carcinogenic effects observed in rodents may not translate to humans. Some key points:

- Epidemiological studies in humans receiving chronic therapy with proton pump inhibitors like omeprazole did not show similar findings of increased gastric tumors seen in rats (source 3)
- The rat findings are thought to be related to increased gastrin levels causing hyperplasia of gastric cells, a mechanism that does not appear to occur in humans (source 3)
- While omeprazole was flagged as a potential genotoxic carcinogen based on computer modeling (source 5), other studies showed no increased cancer risk in humans treated with the related drug metronidazole despite its mutagenicity in experimental studies (source 1)